In [ ]:
/* def main(args: Array[String]): Unit = {
   val sc = new SparkContext(new SparkConf().setAppName("DoubanRecommender"))
    val DataHome = if (args.length > 0) args(0) else "file:///Users/hadoop/eda/data/douban/"
    val rawUserMoviesData = sc.textFile(DataHome + "user_movies.csv")
} */

In [2]:
//val sc = new SparkContext(new SparkConf().setAppName("DoubanRecommender"))
//val DataHome = if (args.length > 0) args(0) else "file:///Users/hadoop/eda/data/douban/"

Name: Compile Error
Message: <console>:19: error: not found: value args
       val DataHome = if (args.length > 0) args(0) else "file:///Users/hadoop/eda/data/douban/"
                          ^
<console>:19: error: not found: value args
       val DataHome = if (args.length > 0) args(0) else "file:///Users/hadoop/eda/data/douban/"
                                           ^
StackTrace: 

In [14]:
import scala.collection.Map
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.SparkContext._
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.recommendation._
import org.apache.spark.rdd.RDD

In [4]:
val DataHome = "file:///Users/hadoop/eda/data/douban/"

In [5]:
DataHome

file:///Users/hadoop/eda/data/douban/

In [6]:
val rawUserMoviesData = sc.textFile(DataHome + "user_movies.csv")

In [7]:
rawUserMoviesData

file:///Users/hadoop/eda/data/douban/user_movies.csv MapPartitionsRDD[2] at textFile at <console>:22

In [18]:
rawUserMoviesData.first

adamwzw,20645098,4

In [29]:
rawUserMoviesData.cache

file:///Users/hadoop/eda/data/douban/user_movies.csv MapPartitionsRDD[2] at textFile at <console>:22

In [37]:
rawUserMoviesData.map(_.split(',')(0).trim).distinct().zipWithUniqueId().map(_._2.toDouble).stats()

(count: 223239, mean: 111620.188663, stdev: 64445.607152, max: 223966.000000, min: 0.000000)

In [38]:
rawUserMoviesData.map(_.split(',')(1).trim.toDouble).distinct().stats()

(count: 165, mean: 20734733.139394, stdev: 8241677.225813, max: 26599083.000000, min: 1866473.000000)

In [44]:
rawUserMoviesData.map(_.split(',')(1).trim.toDouble).distinct().stats()

(count: 165, mean: 20734733.139394, stdev: 8241677.225813, max: 26599083.000000, min: 1866473.000000)

In [8]:
val rawHotMoviesData = sc.textFile(DataHome + "hot_movies.csv")

In [9]:
rawHotMoviesData

file:///Users/hadoop/eda/data/douban/hot_movies.csv MapPartitionsRDD[4] at textFile at <console>:22

In [19]:
rawHotMoviesData.first

20645098,8.2,小王子

In [15]:
def buildMovies(rawHotMoviesData: RDD[String]): Map[Int, String] =
rawHotMoviesData.flatMap {
line => val tokens = line.split(',')
if (tokens(0).isEmpty){
None
} else {
Some((tokens(0).toInt, tokens(2)))
}
}.collectAsMap()

In [16]:
def preparation(rawUserMoviesData: RDD[String],
                rawHotMoviesData: RDD[String]
) = {
val userIDStats = rawUserMoviesData.map(_.split(',')(0).trim).distinct().zipWithUniqueId().map(_._2.toDouble).stats()
val itemIDStats = rawUserMoviesData.map(_.split(',')(1).trim.toDouble).distinct().stats()
println(userIDStats)
println(itemIDStats)
val moviesAndName = buildMovies(rawHotMoviesData)
val (movieID, movieName) = moviesAndName.head
println(movieID + " -> " + movieName)
}

In [17]:
preparation(rawUserMoviesData, rawHotMoviesData)

(count: 223239, mean: 111620.188663, stdev: 64445.607152, max: 223966.000000, min: 0.000000)
(count: 165, mean: 20734733.139394, stdev: 8241677.225813, max: 26599083.000000, min: 1866473.000000)
6866928 -> 进击的巨人真人版：前篇


In [47]:
case class MovieRating(userID: String, movieID: Int, rating: Double) extends
     scala.Serializable

In [50]:
def buildRatings(rawUserMoviesData: RDD[String]): RDD[MovieRating] = {
    rawUserMoviesData.map {
        line =>
                val Array(userID, moviesID, countStr) = line.split(',').map(_.trim)
                var count = countStr.toInt
                count = if (count == -1) 3 else count
                MovieRating(userID, moviesID.toInt, count)
    }
}

In [53]:
// Check result of recommendation to one user
def checkRecommenderResult(userID: Int, rawUserMoviesData: RDD[String],
                          bMoviesAndName: Broadcast[Map[Int, String]],
                          reverseUserIDMapping: RDD[(Long, String)],
                          model: MatrixFactorizationModel): Unit ={
    val userName = reverseUserIDMapping.lookup(userID).head
    val recommendations = model.recommendProducts(userID, 5)
    // The recommendation sets to the user 5.
    val recommendedMovieIDs = recommendations.map(_.product).toSet
    
    // Get the movies set which the user 5 has played.
    val rawMoviesForUser = rawUserMoviesData.map(_.split(',')).filter{
    case Array(user, _, _) => user.trim == userName}
    
    val existingUserMovieIDs = rawMoviesForUser.map {
    case Array(_, movieID, _) => movieID.toInt
    }.collect().toSet
    
    println("Recommend to user " + userName + " the movie name:")
    //Recommendating movies name
    bMoviesAndName.value.filter {
    case (id, name) => recommendedMovieIDs.contains(id)
    }.values.foreach(println)
}

In [56]:
def unpersist(model: MatrixFactorizationModel): Unit = {
    // At the moment, it's necessary to manually unpersist the RDDs inside the model
    // when done with it in order to make sure they are promptly uncached
    model.userFeatures.unpersist()
    model.productFeatures.unpersist()
}

In [57]:
def model(sc: SparkContext, 
          rawUserMoviesData: RDD[String],
          rawHotMoviesData: RDD[String]): Unit = {
    val moviesAndName = buildMovies(rawHotMoviesData)
    val bMoviesAndName = sc.broadcast(moviesAndName)
    val data = buildRatings(rawUserMoviesData)
    val userIdToInt: RDD[(String, Long)] = 
        data.map(_.userID).distinct().zipWithUniqueId()
    val reverseUserIDMapping: RDD[(Long, String)] =
        userIdToInt map {case (l, r) => (r, l)}
    val userIDMap: Map[String, Int] = userIdToInt.collectAsMap().map{
        case (n, l) => (n, l.toInt)
        }
    val bUserIDMap = sc.broadcast(userIDMap)
    val ratings: RDD[Rating] = data.map {
        r => Rating(bUserIDMap.value.get(r.userID).get, r.movieID, r.rating)
        }.cache()
        
    //Build model with Collabrative Filtering
    val model = ALS.train(ratings, 50, 10, 0.0001)
    ratings.unpersist()
    println("The First User Feature is Output: ")
    println(model.userFeatures.mapValues(_.mkString(",")).first())
    for (userID <- Array(100, 1001, 10001, 100001, 110000)){
        checkRecommenderResult(userID, rawUserMoviesData, 
        bMoviesAndName, reverseUserIDMapping, model)
    }
    
    unpersist(model)
}

In [58]:
model(sc, rawUserMoviesData, rawHotMoviesData)

The First User Feature is Output: 
(0,-0.2707848846912384,-0.8239490985870361,-0.2288040816783905,-1.3201241493225098,-0.25197556614875793,-0.7580539584159851,0.39071187376976013,1.3047521114349365,-0.5541315674781799,-1.0833860635757446,0.21554742753505707,0.3992951810359955,-1.3039758205413818,-0.8273643851280212,0.2698359489440918,0.9498853087425232,0.7356423735618591,0.16813980042934418,0.15691663324832916,1.1658538579940796,-0.9022631645202637,-0.7636549472808838,-0.5596442818641663,0.7390807271003723,0.9053812026977539,0.6829679012298584,0.4719841182231903,0.24569830298423767,-0.6468056440353394,2.3752706050872803,-0.631567656993866,-0.8765032291412354,-1.2385278940200806,0.808280348777771,-1.1208305358886719,-2.5614330768585205,-0.8047855496406555,-1.3208379745483398,0.5838742852210999,-0.3246144950389862,-0.3470260798931122,0.343004435300827,-0.41452157497406006,-0.7579929828643799,0.4793323874473572,-0.6534832715988159,0.17293120920658112,1.6464449167251587,-0.7856852412223816

In [65]:
def recommend(sc: SparkContext,
                rawUserMoviesData: RDD[String],
                rawHotMoviesData: RDD[String],
                base:String): Unit = {
    val moviesAndName = buildMovies(rawHotMoviesData)
    val bMoviesAndName = sc.broadcast(moviesAndName)

    val data = buildRatings(rawUserMoviesData)

    val userIdToInt: RDD[(String, Long)] =
      data.map(_.userID).distinct().zipWithUniqueId()
    val reverseUserIDMapping: RDD[(Long, String)] =
      userIdToInt map { case (l, r) => (r, l) }

    val userIDMap: Map[String, Int] =
      userIdToInt.collectAsMap().map { case (n, l) => (n, l.toInt) }

    val bUserIDMap = sc.broadcast(userIDMap)
    val bReverseUserIDMap = sc.broadcast(reverseUserIDMapping.collectAsMap())

    val ratings: RDD[Rating] = data.map { r =>
      Rating(bUserIDMap.value.get(r.userID).get, r.movieID, r.rating)
    }.cache()
    //使用协同过滤算法建模
    //val model = ALS.trainImplicit(ratings, 10, 10, 0.01, 1.0)
    val model = ALS.train(ratings, 50, 10, 0.0001)
    ratings.unpersist()

    model.save(sc, base+"model")
    //val sameModel = MatrixFactorizationModel.load(sc, base + "model")

    val allRecommendations = model.recommendProductsForUsers(5) map {
      case (userID, recommendations) => {
        var recommendationStr = ""
        for (r <- recommendations) {
          recommendationStr += r.product + ":" + bMoviesAndName.value.getOrElse(r.product, "") + ","
        }
        if (recommendationStr.endsWith(","))
          recommendationStr = recommendationStr.substring(0,recommendationStr.length-1)

        (bReverseUserIDMap.value.get(userID).get,recommendationStr)
      }
    }

    //allRecommendations.saveAsTextFile(base + "result.csv")
    allRecommendations.coalesce(1).sortByKey().saveAsTextFile(base + "result.csv")

    unpersist(model)
  }

In [66]:
recommend(sc, rawUserMoviesData, rawHotMoviesData, DataHome)

Name: java.lang.RuntimeException
Message: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
StackTrace: org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
org.apache.spark.sql.hive.client.ClientWrapper.<init>(ClientWrapper.scala:204)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:249)
org.apache.spark.sql.hive.HiveContext.metadataHive$lzycompute(HiveContext.scala:327)
org.apache.spark.sql.hive.HiveContext.metadataHive(HiveContext.scala:237)
org.apache.spark.sql.hive.HiveContext$$anon$2.<init>(HiveContext.scala:457)
org.apache.spark.